In [1]:
from moseq2_detectron_extract.io.session import Session
import matplotlib.pyplot as plt
import numpy as np
import cv2
import seaborn as sns

from moseq2_detectron_extract.proc.proc import prep_raw_frames, clean_frames, get_frame_features, scale_raw_frames
from moseq2_detectron_extract.proc.hha_encode_reference import getHHA, getCameraParam
from moseq2_detectron_extract.proc.fast_hha_encode import HHAEncoder
from moseq2_detectron_extract.proc.roi import plane_ransac, apply_roi
from moseq2_detectron_extract.io.annot import read_annotations
from moseq2_detectron_extract.io.image import write_image

from scipy import signal
from scipy import ndimage
import math
from mpl_toolkits.axes_grid1 import ImageGrid
import itertools
import re
import os
import json
from tqdm import tqdm
from matplotlib.patches import Rectangle

import glob
from moseq2_detectron_extract.io.image import read_image

def plot_tri_channel(frame, labels=None):
    if labels is None:
        labels = []

    fig = plt.figure(figsize=(24, 6))
    grid = ImageGrid(fig, 111, nrows_ncols=(1, 4), cbar_mode='each', axes_pad=0.4, share_all=True,)

    for i, (ax, cax, label) in enumerate(itertools.zip_longest(grid, grid.cbar_axes, labels, fillvalue="")):
        if i < 3:
            im = ax.imshow(frame[:, :, i])
            cax.colorbar(im)
        else:
            im = ax.imshow(frame)

        ax.set_title(label)

In [7]:
#annot_file = "C:\\Users\\thackray\\moseq-detectron-data\\2022-02-19_sci-dataset-outlier-annotated-keypoints-masks\\project-64-at-2022-02-19-19-06-e5e40b7e.json"
annot_file = r"C:\Users\thackray\moseq-detectron-data\2021-09-15_sci-dataset-big-annotated-keypoints-masks\project-3-at-2022-06-13-22-03-09402624.json"
annot_dir = os.path.dirname(annot_file)

annots = read_annotations(annot_file)
#annots = sorted(annots, key=lambda a: a['file_name'])
pattern = re.compile(r"(session_\d+)_depth_(\d+).png")
session_frames = {}
for a in tqdm(annots, desc='Annotations'):
    session_id, frame_idx = pattern.search(a['file_name']).groups()

    if session_id not in session_frames:
        session_frames[session_id] = []
    session_frames[session_id].append(int(frame_idx))

for k, v in session_frames.items():
    session_frames[k] = sorted(v)

#print(session_frames)
#break



for s, (session_id, selected_frame_idxs) in enumerate(session_frames.items()):
    print(f'Working on {session_id} ({s+1}/{len(session_frames.keys())})')

    info_dir = os.path.join(annot_dir, '.info', session_id)
    with open(os.path.join(info_dir, 'info.json'), mode='r') as info_file:
        info = json.load(info_file)

    in_file = list(filter(lambda f: str(f).count(session_id) > 0, info['parameters']['input_file']))[0]
    in_file = in_file.replace('E:\\moseq_raw_data\\SCI-BL6', 'U:\\MoSeq\\moseq2\\Raw_Data\\Abraira_Lab\\Spinal_Cord_Injury-C57BL6')
    if not os.path.exists(in_file):
        print(f'Cannot find {session_id}!')
        continue
    session = Session(in_file)
    first_frame, bground_im, roi, true_depth = session.find_roi(cache_dir=info_dir)
    crop_bground_im = apply_roi(bground_im[None,:,:], roi)[0]

    encoder = HHAEncoder(crop_bground_im)

    count = 0
    for b, (frame_idxs, raw_frames) in enumerate(session.iterate()):
        for j in range(raw_frames.shape[0]):
            if frame_idxs[j] in selected_frame_idxs:
                count += 1
                print(f' -> Producing frame {frame_idxs[j]} ({count}/{len(selected_frame_idxs)})')
                rf = prep_raw_frames(raw_frames[j][None, :, :], bground_im=None, roi=roi, vmin=None, vmax=None)[0]
                hha = encoder.encode(rf)

                name = f'{session_id}_mda_{frame_idxs[j]}.png'
                dest = os.path.join(annot_dir, 'images', name)
                write_image(dest, hha, scale=False, dtype='uint8')



Annotations: 100%|██████████| 2100/2100 [00:00<00:00, 524850.34it/s]


Working on session_20200814115819 (1/21)


 -> Producing frame 261 (1/100)
 -> Producing frame 345 (2/100)
 -> Producing frame 1344 (3/100)
 -> Producing frame 2947 (4/100)
 -> Producing frame 3047 (5/100)
 -> Producing frame 3278 (6/100)
 -> Producing frame 3537 (7/100)
 -> Producing frame 3742 (8/100)
 -> Producing frame 3868 (9/100)
 -> Producing frame 4758 (10/100)
 -> Producing frame 5060 (11/100)
 -> Producing frame 5287 (12/100)
 -> Producing frame 5712 (13/100)
 -> Producing frame 6159 (14/100)
 -> Producing frame 6383 (15/100)
 -> Producing frame 7218 (16/100)
 -> Producing frame 7511 (17/100)
 -> Producing frame 7680 (18/100)
 -> Producing frame 8442 (19/100)
 -> Producing frame 8580 (20/100)
 -> Producing frame 8699 (21/100)
 -> Producing frame 8755 (22/100)
 -> Producing frame 9249 (23/100)
 -> Producing frame 9313 (24/100)
 -> Producing frame 9702 (25/100)
 -> Producing frame 9788 (26/100)
 -> Producing frame 10229 (27/100)
 -> Producing frame 11263 (28/100)
 -> Producing frame 11782 (29/100)
 -> Producing frame 11

 -> Producing frame 118 (1/100)
 -> Producing frame 219 (2/100)
 -> Producing frame 1073 (3/100)
 -> Producing frame 1187 (4/100)
 -> Producing frame 1437 (5/100)
 -> Producing frame 1666 (6/100)
 -> Producing frame 2556 (7/100)
 -> Producing frame 2742 (8/100)
 -> Producing frame 3055 (9/100)
 -> Producing frame 3152 (10/100)
 -> Producing frame 3339 (11/100)
 -> Producing frame 3364 (12/100)
 -> Producing frame 3764 (13/100)
 -> Producing frame 4329 (14/100)
 -> Producing frame 4604 (15/100)
 -> Producing frame 4899 (16/100)
 -> Producing frame 5418 (17/100)
 -> Producing frame 6671 (18/100)
 -> Producing frame 6849 (19/100)
 -> Producing frame 6905 (20/100)
 -> Producing frame 7149 (21/100)
 -> Producing frame 7674 (22/100)
 -> Producing frame 8017 (23/100)
 -> Producing frame 8038 (24/100)
 -> Producing frame 8094 (25/100)
 -> Producing frame 8276 (26/100)
 -> Producing frame 8962 (27/100)
 -> Producing frame 9566 (28/100)
 -> Producing frame 10042 (29/100)
 -> Producing frame 1016

 -> Producing frame 187 (1/100)
 -> Producing frame 301 (2/100)
 -> Producing frame 403 (3/100)
 -> Producing frame 972 (4/100)
 -> Producing frame 1115 (5/100)
 -> Producing frame 2033 (6/100)
 -> Producing frame 2599 (7/100)
 -> Producing frame 2730 (8/100)
 -> Producing frame 2866 (9/100)
 -> Producing frame 3085 (10/100)
 -> Producing frame 3184 (11/100)
 -> Producing frame 3352 (12/100)
 -> Producing frame 3851 (13/100)
 -> Producing frame 4050 (14/100)
 -> Producing frame 4820 (15/100)
 -> Producing frame 4994 (16/100)
 -> Producing frame 5438 (17/100)
 -> Producing frame 5458 (18/100)
 -> Producing frame 5478 (19/100)
 -> Producing frame 5772 (20/100)
 -> Producing frame 5782 (21/100)
 -> Producing frame 6000 (22/100)
 -> Producing frame 7022 (23/100)
 -> Producing frame 7415 (24/100)
 -> Producing frame 7823 (25/100)
 -> Producing frame 8525 (26/100)
 -> Producing frame 8665 (27/100)
 -> Producing frame 8871 (28/100)
 -> Producing frame 9019 (29/100)
 -> Producing frame 9116 (3

...

 -> Producing frame 429 (1/100)
 -> Producing frame 588 (2/100)
 -> Producing frame 821 (3/100)
 -> Producing frame 835 (4/100)
 -> Producing frame 871 (5/100)
 -> Producing frame 1283 (6/100)
 -> Producing frame 1523 (7/100)
 -> Producing frame 1837 (8/100)
 -> Producing frame 2019 (9/100)
 -> Producing frame 2095 (10/100)
 -> Producing frame 2409 (11/100)
 -> Producing frame 2771 (12/100)
 -> Producing frame 3380 (13/100)
 -> Producing frame 3526 (14/100)
 -> Producing frame 3708 (15/100)
 -> Producing frame 4348 (16/100)
 -> Producing frame 5143 (17/100)
 -> Producing frame 5218 (18/100)
 -> Producing frame 5285 (19/100)
 -> Producing frame 6319 (20/100)
 -> Producing frame 6486 (21/100)
 -> Producing frame 6535 (22/100)
 -> Producing frame 7059 (23/100)
 -> Producing frame 7156 (24/100)
 -> Producing frame 7365 (25/100)
 -> Producing frame 7797 (26/100)
 -> Producing frame 7816 (27/100)
 -> Producing frame 8506 (28/100)
 -> Producing frame 8558 (29/100)
 -> Producing frame 9432 (30

 -> Producing frame 27 (1/100)
 -> Producing frame 89 (2/100)
 -> Producing frame 801 (3/100)
 -> Producing frame 855 (4/100)
 -> Producing frame 1067 (5/100)
 -> Producing frame 1799 (6/100)
 -> Producing frame 1830 (7/100)
 -> Producing frame 2040 (8/100)
 -> Producing frame 2422 (9/100)
 -> Producing frame 2688 (10/100)
 -> Producing frame 2927 (11/100)
 -> Producing frame 3033 (12/100)
 -> Producing frame 3305 (13/100)
 -> Producing frame 4326 (14/100)
 -> Producing frame 5281 (15/100)
 -> Producing frame 5788 (16/100)
 -> Producing frame 6036 (17/100)
 -> Producing frame 6232 (18/100)
 -> Producing frame 6287 (19/100)
 -> Producing frame 6613 (20/100)
 -> Producing frame 6856 (21/100)
 -> Producing frame 7346 (22/100)
 -> Producing frame 7756 (23/100)
 -> Producing frame 8244 (24/100)
 -> Producing frame 8437 (25/100)
 -> Producing frame 8601 (26/100)
 -> Producing frame 8898 (27/100)
 -> Producing frame 9345 (28/100)
 -> Producing frame 9525 (29/100)
 -> Producing frame 9684 (30/

 -> Producing frame 91 (1/100)
 -> Producing frame 174 (2/100)
 -> Producing frame 486 (3/100)
 -> Producing frame 641 (4/100)
 -> Producing frame 747 (5/100)
 -> Producing frame 1077 (6/100)
 -> Producing frame 1104 (7/100)
 -> Producing frame 1445 (8/100)
 -> Producing frame 1486 (9/100)
 -> Producing frame 1537 (10/100)
 -> Producing frame 2114 (11/100)
 -> Producing frame 2148 (12/100)
 -> Producing frame 2429 (13/100)
 -> Producing frame 2517 (14/100)
 -> Producing frame 3006 (15/100)
 -> Producing frame 3772 (16/100)
 -> Producing frame 3951 (17/100)
 -> Producing frame 4084 (18/100)
 -> Producing frame 5333 (19/100)
 -> Producing frame 5442 (20/100)
 -> Producing frame 6048 (21/100)
 -> Producing frame 6109 (22/100)
 -> Producing frame 6223 (23/100)
 -> Producing frame 6333 (24/100)
 -> Producing frame 6612 (25/100)
 -> Producing frame 7401 (26/100)
 -> Producing frame 7725 (27/100)
 -> Producing frame 7893 (28/100)
 -> Producing frame 7952 (29/100)
 -> Producing frame 8259 (30/

 -> Producing frame 71 (1/100)
 -> Producing frame 589 (2/100)
 -> Producing frame 1100 (3/100)
 -> Producing frame 1147 (4/100)
 -> Producing frame 1387 (5/100)
 -> Producing frame 1672 (6/100)
 -> Producing frame 1863 (7/100)
 -> Producing frame 2154 (8/100)
 -> Producing frame 2274 (9/100)
 -> Producing frame 2711 (10/100)
 -> Producing frame 2959 (11/100)
 -> Producing frame 3456 (12/100)
 -> Producing frame 4126 (13/100)
 -> Producing frame 4243 (14/100)
 -> Producing frame 4433 (15/100)
 -> Producing frame 4563 (16/100)
 -> Producing frame 4917 (17/100)
 -> Producing frame 5223 (18/100)
 -> Producing frame 5695 (19/100)
 -> Producing frame 5879 (20/100)
 -> Producing frame 6769 (21/100)
 -> Producing frame 6978 (22/100)
 -> Producing frame 7064 (23/100)
 -> Producing frame 7657 (24/100)
 -> Producing frame 8419 (25/100)
 -> Producing frame 8625 (26/100)
 -> Producing frame 9119 (27/100)
 -> Producing frame 9215 (28/100)
 -> Producing frame 9447 (29/100)
 -> Producing frame 10177 

 -> Producing frame 399 (1/100)
 -> Producing frame 1126 (2/100)
 -> Producing frame 1355 (3/100)
 -> Producing frame 1653 (4/100)
 -> Producing frame 1859 (5/100)
 -> Producing frame 2466 (6/100)
 -> Producing frame 2593 (7/100)
 -> Producing frame 2914 (8/100)
 -> Producing frame 2925 (9/100)
 -> Producing frame 3115 (10/100)
 -> Producing frame 3309 (11/100)
 -> Producing frame 3464 (12/100)
 -> Producing frame 3512 (13/100)
 -> Producing frame 3729 (14/100)
 -> Producing frame 4863 (15/100)
 -> Producing frame 4969 (16/100)
 -> Producing frame 5018 (17/100)
 -> Producing frame 5284 (18/100)
 -> Producing frame 5495 (19/100)
 -> Producing frame 5593 (20/100)
 -> Producing frame 6017 (21/100)
 -> Producing frame 7023 (22/100)
 -> Producing frame 7475 (23/100)
 -> Producing frame 7698 (24/100)
 -> Producing frame 7823 (25/100)
 -> Producing frame 7989 (26/100)
 -> Producing frame 8288 (27/100)
 -> Producing frame 8331 (28/100)
 -> Producing frame 8852 (29/100)
 -> Producing frame 9193

In [17]:
raw_frames.shape

(1000, 424, 512)

In [7]:
raw_frames[0][None, :, :].shape
rf = prep_raw_frames(raw_frames[0][None, :, :], bground_im=None, roi=roi, vmin=None, vmax=None)[0]
hha = encoder.encode(rf)

In [ ]:
session = Session(r"E:\moseq_raw_data\detectron2_testing_full\session_20200915132521\depth.dat")
first_frame, bground_im, roi, true_depth = session.find_roi()
crop_bground_im = apply_roi(bground_im[None,:,:], roi)[0]

In [35]:
frames_of_interest = range(18520, 18545)
frame_idxs, raw_frames = next(session.sample(25))
bg_sub_raw_frames = prep_raw_frames(raw_frames.copy(), bground_im=bground_im.copy(), roi=roi, vmin=0, vmax=100)
raw_frames = prep_raw_frames(raw_frames, bground_im=None, roi=roi, vmin=None, vmax=None, dtype='uint16')
print(frame_idxs)


[30223, 4304, 1796, 3003, 7209, 305, 17480, 13400, 285, 2372, 2751, 11435, 25877, 17496, 26874, 35591, 23025, 2718, 35339, 13800, 21976, 13174, 33156, 2600, 12256]


In [24]:


messages = []
items = []
items.extend(glob.glob(r'C:\Users\thackray\moseq-detectron-data\2022-02-19_sci-dataset-outlier-annotated-keypoints-masks\images\*_depth_*.png'))
items.extend(glob.glob(r'C:\Users\thackray\moseq-detectron-data\2021-09-15_sci-dataset-big-annotated-keypoints-masks\images\*_depth_*.png'))
for depth_path in sorted(items):
    mda_path = depth_path.replace('_depth_', '_mda_')
    mda_image = read_image(mda_path, dtype='uint8')
    dth_image = read_image(depth_path, dtype='uint8')
    session_id, frame_idx = pattern.search(depth_path).groups()
    read_image(mda_path)

    #if mda_image.shape != dth_image.shape:
    #    messages.append(f'Size mismatch {mda_image.shape} vs {dth_image.shape} in {session_id} frame {frame_idx}')

len(messages)

0

In [19]:
messages

['Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2793',
 'Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2794',
 'Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2796',
 'Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2797',
 'Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2801',
 'Size mismatch (243, 246, 3) vs (242, 245, 3) in session_20200814133251 frame 2802',
 'Size mismatch (250, 252, 3) vs (246, 247, 3) in session_20200819132625 frame 11612',
 'Size mismatch (250, 252, 3) vs (246, 247, 3) in session_20200819132625 frame 14457',
 'Size mismatch (250, 252, 3) vs (246, 247, 3) in session_20200819132625 frame 6610',
 'Size mismatch (250, 252, 3) vs (246, 247, 3) in session_20200819132625 frame 9787',
 'Size mismatch (243, 247, 3) vs (242, 245, 3) in session_20200819132655 frame 4221',
 'Size mismatch (243, 247, 3) vs (242, 245, 3) in se

In [23]:
read_image(r"C:\Users\thackray\moseq-detectron-data\2022-02-19_sci-dataset-outlier-annotated-keypoints-masks\images\session_20200914124549_mda_5881.png")

array([[[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]],

       [[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]],

       [[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]],

       ...,

       [[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]],

       [[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]],

       [[181,   0, 255],
        [181,   0, 255],
        [181,   0, 255],
        ...,
        [181,   0, 255],
        [181,   0, 255],
        [181,   0, 255]]

In [9]:
#annot_file = "C:\\Users\\thackray\\moseq-detectron-data\\2022-02-19_sci-dataset-outlier-annotated-keypoints-masks\\project-64-at-2022-02-19-19-06-e5e40b7e.json"
annot_file = r"C:\Users\thackray\moseq-detectron-data\2021-09-15_sci-dataset-big-annotated-keypoints-masks\project-3-at-2022-06-13-22-03-09402624.json"
annot_dir = os.path.dirname(annot_file)

annots = read_annotations(annot_file)
annots = sorted(annots, key=lambda a: a['file_name'])
pattern = re.compile(r"(session_\d+)_depth_(\d+).png")
session_frames = {}
for a in tqdm(annots, desc='Annotations'):
    session_id, frame_idx = pattern.search(a['file_name']).groups()
    name = f'{session_id}_viz_{frame_idx}.png'
    dest = os.path.join(annot_dir, 'images', name)

    if not os.path.exists(dest):

        depth_path = os.path.join(annot_dir, 'images', os.path.basename(a['file_name']))
        depth_image = read_image(depth_path, dtype='uint8')
        mda_image = read_image(depth_path.replace('_depth_', '_mda_'), dtype='uint8')
        rgb_image = read_image(depth_path.replace('_depth_', '_rgb_'), dtype='uint8')
        imdata = [
            (depth_image, (0, 100)),
            (rgb_image, (None, None)),
            (mda_image, (None, None))
        ]


        fig, axs = plt.subplots(1, 3, figsize=(18,6), sharex=True, sharey=True)
        for i, ax in enumerate(axs):
            im = axs[i].imshow(imdata[i][0], vmin=imdata[i][1][0], vmax=imdata[i][1][1])

            bbox = a['annotations'][0]['bbox']
            axs[i].add_patch(Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], edgecolor='red', facecolor='none'))

        name = f'{session_id}_viz_{frame_idx}.png'
        dest = os.path.join(annot_dir, 'images', name)
        fig.savefig(dest, dpi=300, facecolor='white')
        plt.close(fig)





Annotations: 100%|██████████| 2100/2100 [24:29<00:00,  1.43it/s]
